### Dataset Description

#### Dataset Proposed by Kristina   Store Sales - Time Series Forecasting

https://kaggle.com/competitions/store-sales-time-series-forecasting

### Dataset Description


In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

#### File Descriptions and Data Field Information


#### train.csv


- The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
- store_nbr identifies the store at which the products are sold.
- family identifies the type of product sold.
- sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
- onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.


#### test.csv


- The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
- The dates in the test data are for the 15 days after the last date in the training data.
 

#### stores.csv


- Store metadata, including city, state, type, and cluster.
- cluster is a grouping of similar stores.


#### oil.csv


-Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)


#### holidays_events.csv


- Holidays and Events, with metadata
- NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
- Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).
- 
Additional Notes

- Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [137]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns  


In [138]:
import os


In [139]:
path = './data_favorita/'
os.listdir(path)

data_oil=pd.read_csv(path+'oil.csv')
data_holidays_events=pd.read_csv(path+'holidays_events.csv')
data_stores=pd.read_csv(path+'stores.csv')
data_train=pd.read_csv(path+'train.csv',parse_dates=['date'])
#data_test=pd.read_csv(path+'test.csv',parse_dates=['date'])
data_transactions=pd.read_csv(path+'transactions.csv')


In [140]:
data_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [141]:
data_stores.describe([1/3,2/3])

,store_nbr,cluster
count,54.000000,54.000000
mean,27.500000,8.481481
std,15.732133,4.693395
min,1.000000,1.000000
33.3%,18.666667,6.000000
50%,27.500000,8.500000
66.7%,36.333333,11.000000
max,54.000000,17.000000


In [142]:
data_stores.isnull().sum()

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

In [143]:
data_stores.duplicated().sum()

np.int64(0)

In [144]:
data_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


#### No duplicates and null found so this looks fairly clean in the first go

#### Now lets look at the transactions data



In [145]:
data_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [146]:
data_transactions.duplicated().sum()

np.int64(0)

In [147]:
data_transactions.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [148]:
data_transactions.isna().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [149]:
data_transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


##### let us change the dtype of the date column

In [150]:
try:
	data_transactions["date"] = pd.to_datetime(data_transactions["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_transactions["date"].describe()

It worked!


count                            83488
mean     2015-05-20 16:07:40.866232064
min                2013-01-01 00:00:00
25%                2014-03-27 00:00:00
50%                2015-06-08 00:00:00
75%                2016-07-14 06:00:00
max                2017-08-15 00:00:00
Name: date, dtype: object

#### Lets join the store and the transactions data


In [151]:
data_transactions_stores=data_transactions.merge(data_stores,on='store_nbr',how='left')

In [152]:
data_transactions_stores.head()

,date,store_nbr,transactions,city,state,type,cluster
0,2013-01-01,25,770,Salinas,Santa Elena,D,1
1,2013-01-02,1,2111,Quito,Pichincha,D,13
2,2013-01-02,2,2358,Quito,Pichincha,D,13
3,2013-01-02,3,3487,Quito,Pichincha,D,8
4,2013-01-02,4,1922,Quito,Pichincha,D,9


#### Now lets look at the holidays and events data

In [153]:
data_holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [154]:
try:
	data_holidays_events["date"] = pd.to_datetime(data_holidays_events["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_holidays_events["date"].describe()

It worked!


count                              350
mean     2015-04-24 00:45:15.428571392
min                2012-03-02 00:00:00
25%                2013-12-23 06:00:00
50%                2015-06-08 00:00:00
75%                2016-07-03 00:00:00
max                2017-12-26 00:00:00
Name: date, dtype: object

In [155]:
data_holidays_events.describe()

,date
count,350
mean,2015-04-24 00:45:15.428571392
min,2012-03-02 00:00:00
25%,2013-12-23 06:00:00
50%,2015-06-08 00:00:00
75%,2016-07-03 00:00:00
max,2017-12-26 00:00:00


In [156]:
data_holidays_events.isnull().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [157]:
data_holidays_events.duplicated().sum()

np.int64(0)

In [158]:
data_holidays_events.isna().sum()

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

In [159]:
data_holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


#### Now lets look at the oil data

In [160]:
data_oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [161]:
data_oil.describe()

,dcoilwtico
count,1175.000000
mean,67.714366
std,25.630476
min,26.190000
25%,46.405000
50%,53.190000
75%,95.660000
max,110.620000


In [162]:
data_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

#### So oIl data has null data and will need cleaning or imputation.Lets analyze the oil file further


In [163]:
data_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


Let us change teh date to date time64 object type

In [164]:
try:
	data_oil["date"] = pd.to_datetime(data_oil["date"], format='%Y-%m-%d')
	print("It worked!")
except ValueError as e:
	print(f"ValueError for AccountOpened: {e}")
data_oil["date"].describe()


It worked!


count                   1218
mean     2015-05-02 12:00:00
min      2013-01-01 00:00:00
25%      2014-03-03 06:00:00
50%      2015-05-02 12:00:00
75%      2016-06-30 18:00:00
max      2017-08-31 00:00:00
Name: date, dtype: object

In [165]:
# Impute missing oil prices with the price from the previous row
#data_oil['dcoilwtico'].fillna(method='ffill', inplace=True)
data_oil["dcoilwtico"].ffill()

0         NaN
1       93.14
2       92.97
3       93.12
4       93.20
        ...  
1213    47.65
1214    46.40
1215    46.46
1216    45.96
1217    47.26
Name: dcoilwtico, Length: 1218, dtype: float64

In [166]:
data_oil.duplicated().sum()

np.int64(0)

In [167]:
data_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [168]:
mean_price = data_oil['dcoilwtico'].mean()
#data_oil['dcoilwtico'].fillna(mean_price, inplace=True)
data_oil.fillna({'dcoilwtico': mean_price}, inplace=True)


In [169]:
data_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

#### so oil data doesnt have any duplicate values and there are no more null values and the date is converted to datetime64 which will et us do mathmatical operations on it

#### Let us now look at the train data


In [170]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


In [171]:
data_train.describe()

,id,date,store_nbr,sales,onpromotion
count,3.000888e+06,3000888,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2015-04-24 08:27:04.703088384,2.750000e+01,3.577757e+02,2.602770e+00
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,2014-02-26 18:00:00,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2015-04-24 12:00:00,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,2016-06-19 06:00:00,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,2017-08-15 00:00:00,5.400000e+01,1.247170e+05,7.410000e+02
std,8.662819e+05,NaN,1.558579e+01,1.101998e+03,1.221888e+01


In [172]:
data_train.isnull().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [173]:
data_train.duplicated().sum()

np.int64(0)

In [174]:
data_train.head(20)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0
5,5,2013-01-01,1,BREAD/BAKERY,0.0,0
6,6,2013-01-01,1,CELEBRATION,0.0,0
7,7,2013-01-01,1,CLEANING,0.0,0
8,8,2013-01-01,1,DAIRY,0.0,0
9,9,2013-01-01,1,DELI,0.0,0


In [175]:
# Convert the 'family' column to integer codes
data_train['family'], _ = pd.factorize(data_train['family'])

# Verify the conversion
data_train.tail(100)

,id,date,store_nbr,family,sales,onpromotion
3000788,3000788,2017-08-15,6,32,47.575,0
3000789,3000789,2017-08-15,7,0,5.000,0
3000790,3000790,2017-08-15,7,1,0.000,0
3000791,3000791,2017-08-15,7,2,11.000,2
3000792,3000792,2017-08-15,7,3,3999.000,9
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,28,438.133,0
3000884,3000884,2017-08-15,9,29,154.553,1
3000885,3000885,2017-08-15,9,30,2419.729,148
3000886,3000886,2017-08-15,9,31,121.000,8


In [187]:
df1 = data_train.merge(data_stores,  
                      on='store_nbr',
                      how='left')

In [188]:
df2 = df1.merge(data_transactions,
                on=['store_nbr', 'date'],
                how='left')
df3 = df2.merge(data_oil,
                on='date',
                how='left')
final_df = df3.merge(data_holidays_events,
                     on='date',
                     how='left')

In [186]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 17 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            int64         
 1   date          datetime64[ns]
 2   store_nbr     int64         
 3   family        int64         
 4   sales         float64       
 5   onpromotion   int64         
 6   city          object        
 7   state         object        
 8   type_x        object        
 9   cluster       int64         
 10  transactions  float64       
 11  dcoilwtico    float64       
 12  type_y        object        
 13  locale        object        
 14  locale_name   object        
 15  description   object        
 16  transferred   object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(8)
memory usage: 396.1+ MB
